# Necessary Common Functions

Those functions should be ran before each part.

In [1]:
import os
import random
import numpy as np
import pandas as pd
import cudf, itertools
import scipy.sparse as ssp
from functools import lru_cache
from tqdm import tqdm, trange
from collections import Counter, defaultdict
import numba 
from numba import jit
import datasets
from datasets import Dataset as TFDataset
import multiprocessing

/root/autodl-tmp/xiaolong/anaconda3/envs/pytorch_gpu/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_sessions(df: pd.DataFrame, test=False, list_item=False) -> list:
    
    all_item = []
    if 'next_item' in df and not test:
        if list_item:
            for i in trange(len(df)):
                all_item.append(np.concatenate([np.array(df.loc[i, 'prev_items']), np.array(df.loc[i, 'next_item'])], axis=0))
        else:
            for i in trange(len(df)):
                all_item.append(eval((df.loc[i, 'prev_items'][:-1]+f" '{df.loc[i, 'next_item']}']").replace(" ", ",")))
    else:
        if list_item:
            all_item = df['prev_items']
        else:
            for i in trange(len(df)):
                all_item.append(eval((df.loc[i, 'prev_items']).replace(" ", ",")))
    return all_item

In [3]:
def get_co_occurence_dict(sessions: list, bidirection: bool=True, weighted: bool=False, max_dis=None) -> dict:
    res = {}
    for sess in tqdm(sessions):
        for i, id in enumerate(sess):
            if id not in res:
                res[id] = Counter()
            
            if max_dis == None:
                e = len(sess)
            else:
                e = min(i + max_dis + 1, len(sess))

            for j in range(i+1, e):
                if not weighted:
                    res[id][sess[j]] += 1
                else:
                    res[id][sess[j]] += 1 / (j-i)
                if bidirection:
                    if sess[j] not in res:
                        res[sess[j]] = Counter()
                    if not weighted:
                        res[sess[j]][id] += 1
                    else:
                        res[sess[j]][id] += 1 / (j-i)
    return res

In [4]:
def sort_co_occurence_dict(co_occurence_dict: dict) -> dict:
    res = {}
    for k,v in co_occurence_dict.items():
        res[k] = dict(sorted(v.items(), key=lambda item: -item[1]))
    return res

In [5]:
def cast_dtype(df : pd.DataFrame):
    for k in df.columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

In [6]:
def get_session_last_item(session_df):
    last_items = []
    num_sessions = len(session_df)
    for i in tqdm(range(num_sessions)):
        sess = session_df.iloc[i]
        sess_prev_items = sess['prev_items']
        
        product_list = sess_prev_items.strip('[]').split(' ')
        last_item = product_list[-1].strip("'\n")

        last_items.append(last_item)
    return last_items 

In [7]:
def get_co_graph_counts(session_last_items, merged_candidates_df, co_graph_dict):
    co_graph_count_list = []
    for idx, row in tqdm(merged_candidates_df.iterrows(), total=merged_candidates_df.shape[0]):
        sess_id = row['sess_id']
        product = row['product']
        last_item = session_last_items[sess_id]
        co_graph_count = co_graph_dict[last_item][product]
        co_graph_count_list.append(co_graph_count)
    return co_graph_count_list

In [8]:
def flatten_co_graph_dict(co_graph_dict):
    product_list = []
    neighbor_list = []
    counts_list = []
    for product in tqdm(co_graph_dict.keys(), total=len(co_graph_dict)):
        for neigh in co_graph_dict[product].keys():
            product_list.append(product)
            neighbor_list.append(neigh)
            counts_list.append(co_graph_dict[product][neigh])
    return pd.DataFrame({'product_' : product_list, 'neighbor' : neighbor_list, 'counts' : counts_list})

In [9]:
def normalize_co_graph_counts(merged_candidates_counts):
    # normalize co graph counts 
    merged_candidates_counts_g = cudf.from_pandas(merged_candidates_counts)
    sessions_count_sum_g = merged_candidates_counts_g[['sess_id', 'counts']].groupby('sess_id').sum()
    sessions_count_sum_g.sort_index(inplace=True)

    sessions_count_sum = sessions_count_sum_g.to_pandas()

    candidates_count_sum = sessions_count_sum.loc[merged_candidates_counts['sess_id']].reset_index(drop=True)
    merged_candidates_counts['counts_cum'] = candidates_count_sum['counts']
    merged_candidates_counts['normalized_counts'] = merged_candidates_counts['counts'] / merged_candidates_counts['counts_cum']

    del merged_candidates_counts_g
    del sessions_count_sum_g
    

# Merge valid co-graph counts 

In [10]:
merged_candidates_feature_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_no_hist_feature.parquet'
train_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_train_sessions.csv'
valid_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_valid_sessions.csv'
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1.csv'
product_data_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/products_train.csv'

In [11]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature():
    return pd.read_parquet(merged_candidates_feature_path)

@lru_cache(maxsize=1)
def read_product_data():
    return pd.read_csv(product_data_path)

@lru_cache(maxsize=1)
def read_train_data():
    return pd.read_csv(train_sessions_path)

@lru_cache(maxsize=1)
def read_valid_data():
    return pd.read_csv(valid_sessions_path)

@lru_cache(maxsize=1)
def read_test_data():
    return pd.read_csv(test_sessions_path)

In [12]:
merged_candidates_feature = read_merged_candidates_feature()

In [13]:
merged_candidates = merged_candidates_feature[['sess_id', 'sess_locale', 'product']]

In [14]:
train_sess_data = read_train_data()
valid_sess_data = read_valid_data()
test_sess_data = read_test_data()
product = read_product_data()

In [15]:
train_sess_item = get_sessions(train_sess_data, list_item=False)
valid_sess_item = get_sessions(valid_sess_data, test=True, list_item=False)

100%|██████████| 361581/361581 [00:41<00:00, 8810.07it/s] 


## bidirection

In [16]:
# bidirection
co_occurence_dict_bi = get_co_occurence_dict(train_sess_item + valid_sess_item, bidirection=True, weighted=False)

100%|██████████| 3919479/3919479 [03:22<00:00, 19329.73it/s]


In [17]:
# only one arg, can't use another arg
def get_bi_valid_session_co_graph_candidates(sess_id):
    sess = valid_sess_item[sess_id]
    prev_items = set()
    cand_counter = Counter()
    for item in sess:
        if item in co_occurence_dict_bi and item not in prev_items:
            cand_counter = cand_counter + co_occurence_dict_bi[item]
            prev_items.add(item) # one time for every item
    for item in sess:
        if item in cand_counter:
            cand_counter.pop(item) # remove history items 

    return cand_counter

In [18]:
valid_co_graph_candidates = TFDataset.from_dict({'sess_id' : list(range(valid_sess_data.shape[0]))})

In [19]:
# about 1 mins
sess_ids = list(range(len(valid_sess_item)))
pool = multiprocessing.Pool(10)
bi_valid_sessions_counter = pool.map(get_bi_valid_session_co_graph_candidates, sess_ids)

In [31]:
valid_co_graph_candidates, len(bi_valid_sessions_counter)

(Dataset({
     features: ['sess_id'],
     num_rows: 361581
 }),
 361581)

In [32]:
all_items_co_graph_count_list = []
for row in tqdm(merged_candidates.itertuples(), total=merged_candidates.shape[0]):
    all_items_co_graph_count_list.append(bi_valid_sessions_counter[row.sess_id][row.product])
assert len(all_items_co_graph_count_list) == merged_candidates.shape[0]
merged_candidates['all_items_co_graph_count_0'] = all_items_co_graph_count_list

100%|██████████| 84407339/84407339 [04:54<00:00, 286783.36it/s]
/tmp/ipykernel_579818/4091948514.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['all_items_co_graph_count_0'] = all_items_co_graph_count_list


In [33]:
count_sum_array = merged_candidates.groupby(by='sess_id')['all_items_co_graph_count_0'].sum().to_numpy()
assert len(count_sum_array[merged_candidates['sess_id']]) == merged_candidates.shape[0]
merged_candidates['normalized_all_items_co_graph_count_0'] = count_sum_array[merged_candidates['sess_id']]
merged_candidates['normalized_all_items_co_graph_count_0'] = merged_candidates['all_items_co_graph_count_0'] / merged_candidates['normalized_all_items_co_graph_count_0']

/tmp/ipykernel_579818/860000090.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['normalized_all_items_co_graph_count_0'] = count_sum_array[merged_candidates['sess_id']]
/tmp/ipykernel_579818/860000090.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['normalized_all_items_co_graph_count_0'] = merged_candidates['all_items_co_graph_count_0'] / merged_candidates['normalized_all_items_co_graph_count_0']


In [34]:
merged_candidates_feature['normalized_all_items_co_graph_count_0'] = merged_candidates['normalized_all_items_co_graph_count_0']
merged_candidates_feature['all_items_co_graph_count_0'] = merged_candidates['all_items_co_graph_count_0']

In [45]:
merged_candidates_feature.query('sess_id==20000').sort_values(by=['sasrec_normalized_scores_3'], ascending=False)[['product', 'sasrec_normalized_scores_3', 'all_items_co_graph_count_0', 'normalized_all_items_co_graph_count_0']][:25]

,product,sasrec_normalized_scores_3,all_items_co_graph_count_0,normalized_all_items_co_graph_count_0
4672245,B07D98ZNJX,8.362947e-01,11,0.250000
4672246,B07D997JWG,1.596784e-01,10,0.227273
4672267,B07QJB4MFD,2.989717e-03,4,0.090909
4672174,B007SQ3HGS,3.628744e-04,2,0.045455
4672394,B09Y5GC5R7,3.485525e-04,2,0.045455
4672283,B07V3P7Z9R,1.137960e-04,2,0.045455
4672297,B081N7R51C,1.123027e-04,1,0.022727
4672300,B081N8XPQH,6.985810e-05,0,0.000000
4672284,B07V4PBMWT,1.069833e-05,2,0.045455
4672340,B08SVPH33Z,9.188924e-06,3,0.068182


## uni and weighted

In [46]:
# weight 
# co_occurence_dict_bi_weight = get_co_occurence_dict(train_sess_item + valid_sess_item, bidirection=True, weighted=True)
co_occurence_dict_uni_weight = get_co_occurence_dict(train_sess_item + valid_sess_item, bidirection=False, weighted=True)

100%|██████████| 3919479/3919479 [02:20<00:00, 27963.44it/s]


In [47]:
merged_candidates = merged_candidates_feature[['sess_id', 'sess_locale', 'product']]

In [51]:
# only one arg, can't use another arg
def get_uni_wei_valid_session_co_graph_candidates(sess_id):
    sess = valid_sess_item[sess_id]
    prev_items = set()
    cand_counter = Counter()
    for item in sess:
        if item in co_occurence_dict_uni_weight and item not in prev_items:
            cand_counter = cand_counter + co_occurence_dict_uni_weight[item]
            prev_items.add(item) # one time for every item
    for item in sess:
        if item in cand_counter:
            cand_counter.pop(item) # remove history items 
    
    return cand_counter

In [49]:
valid_co_graph_candidates = TFDataset.from_dict({'sess_id' : list(range(valid_sess_data.shape[0]))})

In [52]:
# about 1 mins
sess_ids = list(range(len(valid_sess_item)))
pool = multiprocessing.Pool(20)
uni_wei_valid_sessions_counter = pool.map(get_uni_wei_valid_session_co_graph_candidates, sess_ids)

In [53]:
valid_co_graph_candidates, len(uni_wei_valid_sessions_counter)

(Dataset({
     features: ['sess_id'],
     num_rows: 361581
 }),
 361581)

In [54]:
all_items_co_graph_count_list = []
for row in tqdm(merged_candidates.itertuples(), total=merged_candidates.shape[0]):
    all_items_co_graph_count_list.append(uni_wei_valid_sessions_counter[row.sess_id][row.product])
assert len(all_items_co_graph_count_list) == merged_candidates.shape[0]
merged_candidates['all_items_co_graph_count_1'] = all_items_co_graph_count_list

100%|██████████| 84407339/84407339 [04:50<00:00, 290791.30it/s]
/tmp/ipykernel_579818/1702843849.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['all_items_co_graph_count_1'] = all_items_co_graph_count_list


In [55]:
count_sum_array = merged_candidates.groupby(by='sess_id')['all_items_co_graph_count_1'].sum().to_numpy()
assert len(count_sum_array[merged_candidates['sess_id']]) == merged_candidates.shape[0]
merged_candidates['normalized_all_items_co_graph_count_1'] = count_sum_array[merged_candidates['sess_id']]
merged_candidates['normalized_all_items_co_graph_count_1'] = merged_candidates['all_items_co_graph_count_1'] / merged_candidates['normalized_all_items_co_graph_count_1']

/tmp/ipykernel_579818/1383280374.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['normalized_all_items_co_graph_count_1'] = count_sum_array[merged_candidates['sess_id']]
/tmp/ipykernel_579818/1383280374.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['normalized_all_items_co_graph_count_1'] = merged_candidates['all_items_co_graph_count_1'] / merged_candidates['normalized_all_items_co_graph_count_1']


In [56]:
merged_candidates_feature['normalized_all_items_co_graph_count_1'] = merged_candidates['normalized_all_items_co_graph_count_1']
merged_candidates_feature['all_items_co_graph_count_1'] = merged_candidates['all_items_co_graph_count_1']

In [58]:
merged_candidates_feature.query('sess_id==10000').sort_values(by=['sasrec_normalized_scores_3'], ascending=False)[['product', 'sasrec_normalized_scores_3', 'all_items_co_graph_count_1', 'normalized_all_items_co_graph_count_1']][:25]

,product,sasrec_normalized_scores_3,all_items_co_graph_count_1,normalized_all_items_co_graph_count_1
2334309,B07SC4G9ZH,0.995533,2.000000,0.024338
2334430,B09Q68WNR3,0.000537,7.416667,0.090253
2334460,B0B61SRMWJ,0.000449,1.000000,0.012169
2334338,B0837G47DQ,0.000407,2.500000,0.030422
2334316,B07W6HXHN7,0.000278,0.000000,0.000000
2334429,B09Q5Y88PY,0.000229,0.575000,0.006997
2334238,B01A94M10S,0.000222,0.250000,0.003042
2334311,B07SFR9T9V,0.000215,4.000000,0.048676
2334345,B088V4X7CX,0.000171,7.486111,0.091098
2334255,B073CSPPJV,0.000165,5.283333,0.064293


## uni and dis=1

In [59]:
# max dis = 1 
# co_occurence_dict_bi_dis1 = get_co_occurence_dict(train_sess_item + valid_sess_item, bidirection=True, weighted=False, max_dis=1)
co_occurence_dict_uni_dis1 = get_co_occurence_dict(train_sess_item + valid_sess_item, bidirection=False, weighted=False, max_dis=1)

100%|██████████| 3919479/3919479 [01:25<00:00, 45808.92it/s] 


In [60]:
merged_candidates = merged_candidates_feature[['sess_id', 'sess_locale', 'product']]

In [64]:
# only one arg, can't use another arg
def get_uni_dis1_valid_session_co_graph_candidates(sess_id):
    sess = valid_sess_item[sess_id]
    prev_items = set()
    cand_counter = Counter()
    for item in sess:
        if item in co_occurence_dict_uni_dis1 and item not in prev_items:
            cand_counter = cand_counter + co_occurence_dict_uni_dis1[item]
            prev_items.add(item) # one time for every item
    for item in sess:
        if item in cand_counter:
            cand_counter.pop(item) # remove history items 
    
    return cand_counter

In [62]:
valid_co_graph_candidates = TFDataset.from_dict({'sess_id' : list(range(valid_sess_data.shape[0]))})

In [65]:
# about 1 mins
sess_ids = list(range(len(valid_sess_item)))
pool = multiprocessing.Pool(10)
uni_dis1_valid_sessions_counter = pool.map(get_uni_dis1_valid_session_co_graph_candidates, sess_ids)

In [66]:
valid_co_graph_candidates, len(uni_dis1_valid_sessions_counter)

(Dataset({
     features: ['sess_id'],
     num_rows: 361581
 }),
 361581)

In [67]:
all_items_co_graph_count_list = []
for row in tqdm(merged_candidates.itertuples(), total=merged_candidates.shape[0]):
    all_items_co_graph_count_list.append(uni_dis1_valid_sessions_counter[row.sess_id][row.product])
assert len(all_items_co_graph_count_list) == merged_candidates.shape[0]
merged_candidates['all_items_co_graph_count_2'] = all_items_co_graph_count_list

100%|██████████| 84407339/84407339 [04:43<00:00, 297871.22it/s]
/tmp/ipykernel_579818/3640618043.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['all_items_co_graph_count_2'] = all_items_co_graph_count_list


In [68]:
count_sum_array = merged_candidates.groupby(by='sess_id')['all_items_co_graph_count_2'].sum().to_numpy()
assert len(count_sum_array[merged_candidates['sess_id']]) == merged_candidates.shape[0]
merged_candidates['normalized_all_items_co_graph_count_2'] = count_sum_array[merged_candidates['sess_id']]
merged_candidates['normalized_all_items_co_graph_count_2'] = merged_candidates['all_items_co_graph_count_2'] / merged_candidates['normalized_all_items_co_graph_count_2']

/tmp/ipykernel_579818/2314243688.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['normalized_all_items_co_graph_count_2'] = count_sum_array[merged_candidates['sess_id']]
/tmp/ipykernel_579818/2314243688.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['normalized_all_items_co_graph_count_2'] = merged_candidates['all_items_co_graph_count_2'] / merged_candidates['normalized_all_items_co_graph_count_2']


In [69]:
merged_candidates_feature['normalized_all_items_co_graph_count_2'] = merged_candidates['normalized_all_items_co_graph_count_2']
merged_candidates_feature['all_items_co_graph_count_2'] = merged_candidates['all_items_co_graph_count_2']

In [71]:
merged_candidates_feature.query('sess_id==150000').sort_values(by=['sasrec_normalized_scores_3'], ascending=False)[['product', 'sasrec_normalized_scores_3', 'all_items_co_graph_count_2', 'normalized_all_items_co_graph_count_2']][:25]

,product,sasrec_normalized_scores_3,all_items_co_graph_count_2,normalized_all_items_co_graph_count_2
35011672,B09B31TB29,0.997036,3,0.500000
35011742,B0B6G1NYY3,0.001067,0,0.000000
35011690,B09FZCB5P6,0.000453,1,0.166667
35011741,B0B6FT1VPK,0.000280,0,0.000000
35011685,B09FKW2TQR,0.000188,0,0.000000
35011687,B09FKXH2QG,0.000181,0,0.000000
35011580,B07NS3JSDM,0.000179,0,0.000000
35011561,B075569D3Z,0.000122,0,0.000000
35011577,B07NRLQ5MH,0.000075,0,0.000000
35011691,B09FZCR8RX,0.000060,0,0.000000


In [72]:
cast_dtype(merged_candidates_feature)
merged_candidates_feature.to_parquet(merged_candidates_feature_path, engine='pyarrow')

In [67]:
merged_candidates_feature

,sess_id,sess_locale,product,target,sasrec_scores_2,sasrec_normalized_scores_2,product_freq,gru4rec_scores,gru4rec_normalized_scores,sess_avg_price,...,normalized_co_graph_counts_1,co_graph_counts_2,normalized_co_graph_counts_2,roberta_scores,roberta_normalized_scores,title_BM25_scores,sasrec_scores_3,sasrec_normalized_scores_3,normalized_all_items_co_graph_count_0,all_items_co_graph_count_0
0,0,UK,B000OPPVCS,0.0,11.972421,2.286162e-04,104,6.484859,3.816029e-05,7.388571,...,0.004819,2,0.004525,265.826630,1.087245e-03,298.915375,10.891474,2.517129e-04,0.002635,2
1,0,UK,B000V599Y2,0.0,13.152878,7.443427e-04,37,4.342063,4.477209e-06,7.388571,...,0.000000,2,0.004525,259.157867,1.380768e-06,111.069756,10.677187,2.031618e-04,0.003953,3
2,0,UK,B0018HH444,0.0,5.606023,3.928400e-07,7,3.220763,1.458925e-06,7.388571,...,0.004819,1,0.002262,257.331421,2.222824e-07,0.000000,6.074605,2.036883e-06,0.001318,1
3,0,UK,B0079JI4DU,0.0,0.000000,1.443945e-09,67,0.000000,5.824698e-08,7.388571,...,0.002410,2,0.004525,0.000000,0.000000e+00,0.000000,0.000000,4.685961e-09,0.002635,2
4,0,UK,B0079JI4EY,0.0,0.000000,1.443945e-09,77,0.000000,5.824698e-08,7.388571,...,0.004819,2,0.004525,0.000000,0.000000e+00,0.000000,0.000000,4.685961e-09,0.002635,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84407334,361580,DE,B0BB7XV97M,0.0,9.117821,6.077226e-05,56,9.268379,1.396883e-05,32.424000,...,0.000000,0,0.000000,263.574158,1.378417e-03,118.126396,9.635838,3.403967e-05,0.003356,2
84407335,361580,DE,B0BB7YSRBX,0.0,9.163816,6.363281e-05,58,7.047796,1.516259e-06,32.424000,...,0.000000,0,0.000000,263.523743,1.310646e-03,124.881615,9.159988,2.115080e-05,0.001678,1
84407336,361580,DE,B0BB7ZMGY8,0.0,11.256460,5.158278e-04,452,9.359167,1.529639e-05,32.424000,...,0.000000,0,0.000000,263.567017,1.368608e-03,124.881615,10.119755,5.522656e-05,0.038591,23
84407337,361580,DE,B0BD4CP7N3,0.0,-3.778687,1.523433e-10,1,-0.593306,7.282568e-10,32.424000,...,0.000000,0,0.000000,265.401611,8.571040e-03,192.540955,-1.612869,4.433373e-10,0.000000,0


In [69]:
merged_candidates_feature.query('sess_id==10000').sort_values(by=['sasrec_normalized_scores_3'], ascending=False)[['sasrec_scores_3', 'sasrec_normalized_scores_3', 'all_items_co_graph_count_0', 'normalized_all_items_co_graph_count_0']]

,sasrec_scores_3,sasrec_normalized_scores_3,all_items_co_graph_count_0,normalized_all_items_co_graph_count_0
2334309,22.575985,9.955325e-01,3,0.008929
2334430,15.050841,5.369416e-04,12,0.035714
2334460,14.871281,4.486885e-04,6,0.017857
2334338,14.773024,4.066983e-04,3,0.008929
2334316,14.391475,2.776948e-04,2,0.005952
...,...,...,...,...
2334215,-8.178933,4.378892e-14,0,0.000000
2334306,-9.413960,1.273500e-14,2,0.005952
2334357,-9.928643,7.611580e-15,4,0.011905
2334409,-10.111639,6.338708e-15,0,0.000000


# Merge test co-graph counts 

In [73]:
merged_candidates_feature_test_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_test_no_hist_feature.parquet'
train_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_train_sessions.csv'
valid_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_valid_sessions.csv'
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1.csv'
product_data_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/products_train.csv'

In [74]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature_test():
    return pd.read_parquet(merged_candidates_feature_test_path)

@lru_cache(maxsize=1)
def read_product_data():
    return pd.read_csv(product_data_path)

@lru_cache(maxsize=1)
def read_train_data():
    return pd.read_csv(train_sessions_path)

@lru_cache(maxsize=1)
def read_valid_data():
    return pd.read_csv(valid_sessions_path)

@lru_cache(maxsize=1)
def read_test_data():
    return pd.read_csv(test_sessions_path)

In [75]:
merged_candidates_feature_test = read_merged_candidates_feature_test()

In [77]:
merged_candidates_test = merged_candidates_feature_test[['sess_id', 'sess_locale', 'product']]

In [78]:
train_sess_data = read_train_data()
valid_sess_data = read_valid_data()
test_sess_data = read_test_data()
product = read_product_data()

In [79]:
train_sess_item = get_sessions(train_sess_data, list_item=False)
valid_sess_item = get_sessions(valid_sess_data, list_item=False)
test_sess_item = get_sessions(test_sess_data, test=True, list_item=False)

100%|██████████| 316971/316971 [00:17<00:00, 17814.50it/s]


# bidirection

In [80]:
# bidirection
co_occurence_dict_bi = get_co_occurence_dict(train_sess_item + valid_sess_item + test_sess_item, bidirection=True, weighted=False)

100%|██████████| 4236450/4236450 [03:57<00:00, 17837.07it/s]


In [81]:
# only one arg, can't use another arg
def get_bi_test_session_co_graph_candidates(sess_id):
    sess = test_sess_item[sess_id]
    prev_items = set()
    cand_counter = Counter()
    for item in sess:
        if item in co_occurence_dict_bi and item not in prev_items:
            cand_counter = cand_counter + co_occurence_dict_bi[item]
            prev_items.add(item) # one time for every item
    for item in sess:
        if item in cand_counter:
            cand_counter.pop(item) # remove history items 
    
    return cand_counter

In [82]:
test_co_graph_candidates = TFDataset.from_dict({'sess_id' : list(range(test_sess_data.shape[0]))})

In [84]:
# about 1 mins
sess_ids = list(range(len(test_sess_item)))
pool = multiprocessing.Pool(10)
bi_test_sessions_counter = pool.map(get_bi_test_session_co_graph_candidates, sess_ids)

In [85]:
test_co_graph_candidates, len(bi_test_sessions_counter)

(Dataset({
     features: ['sess_id'],
     num_rows: 316971
 }),
 316971)

In [86]:
all_items_co_graph_count_list = []
for row in tqdm(merged_candidates_test.itertuples(), total=merged_candidates_test.shape[0]):
    all_items_co_graph_count_list.append(bi_test_sessions_counter[row.sess_id][row.product])
assert len(all_items_co_graph_count_list) == merged_candidates_test.shape[0]
merged_candidates_test['all_items_co_graph_count_0'] = all_items_co_graph_count_list

100%|██████████| 69428431/69428431 [04:22<00:00, 264832.67it/s]
/tmp/ipykernel_579818/565594440.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates_test['all_items_co_graph_count_0'] = all_items_co_graph_count_list


In [87]:
count_sum_array = merged_candidates_test.groupby(by='sess_id')['all_items_co_graph_count_0'].sum().to_numpy()
assert len(count_sum_array[merged_candidates_test['sess_id']]) == merged_candidates_test.shape[0]
merged_candidates_test['normalized_all_items_co_graph_count_0'] = count_sum_array[merged_candidates_test['sess_id']]
merged_candidates_test['normalized_all_items_co_graph_count_0'] = merged_candidates_test['all_items_co_graph_count_0'] / merged_candidates_test['normalized_all_items_co_graph_count_0']

/tmp/ipykernel_579818/3715222465.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates_test['normalized_all_items_co_graph_count_0'] = count_sum_array[merged_candidates_test['sess_id']]
/tmp/ipykernel_579818/3715222465.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates_test['normalized_all_items_co_graph_count_0'] = merged_candidates_test['all_items_co_graph_count_0'] / merged_candidates_test['normalized_all_items_co_graph_count_0']


In [89]:
merged_candidates_feature_test['normalized_all_items_co_graph_count_0'] = merged_candidates_test['normalized_all_items_co_graph_count_0']
merged_candidates_feature_test['all_items_co_graph_count_0'] = merged_candidates_test['all_items_co_graph_count_0']

## uni and weighted

In [90]:
# weight 
# co_occurence_dict_bi_weight = get_co_occurence_dict(train_sess_item + valid_sess_item + test_sess_item, bidirection=True, weighted=True)
co_occurence_dict_uni_weight = get_co_occurence_dict(train_sess_item + valid_sess_item + test_sess_item, bidirection=False, weighted=True)

100%|██████████| 4236450/4236450 [02:33<00:00, 27510.34it/s]


In [91]:
# only one arg, can't use another arg
def get_uni_wei_test_session_co_graph_candidates(sess_id):
    sess = test_sess_item[sess_id]
    prev_items = set()
    cand_counter = Counter()
    for item in sess:
        if item in co_occurence_dict_uni_weight and item not in prev_items:
            cand_counter = cand_counter + co_occurence_dict_uni_weight[item]
            prev_items.add(item) # one time for every item
    for item in sess:
        if item in cand_counter:
            cand_counter.pop(item) # remove history items 
    
    return cand_counter

In [92]:
test_co_graph_candidates = TFDataset.from_dict({'sess_id' : list(range(test_sess_data.shape[0]))})

In [93]:
# about 1 mins
sess_ids = list(range(len(test_sess_item)))
pool = multiprocessing.Pool(10)
uni_wei_test_sessions_counter = pool.map(get_uni_wei_test_session_co_graph_candidates, sess_ids)

In [94]:
test_co_graph_candidates, len(uni_wei_test_sessions_counter)

(Dataset({
     features: ['sess_id'],
     num_rows: 316971
 }),
 316971)

In [95]:
all_items_co_graph_count_list = []
for row in tqdm(merged_candidates_test.itertuples(), total=merged_candidates_test.shape[0]):
    all_items_co_graph_count_list.append(uni_wei_test_sessions_counter[row.sess_id][row.product])
assert len(all_items_co_graph_count_list) == merged_candidates_test.shape[0]
merged_candidates_test['all_items_co_graph_count_1'] = all_items_co_graph_count_list

100%|██████████| 69428431/69428431 [05:06<00:00, 226645.16it/s]
/tmp/ipykernel_579818/4140053534.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates_test['all_items_co_graph_count_1'] = all_items_co_graph_count_list


In [96]:
count_sum_array = merged_candidates_test.groupby(by='sess_id')['all_items_co_graph_count_1'].sum().to_numpy()
assert len(count_sum_array[merged_candidates_test['sess_id']]) == merged_candidates_test.shape[0]
merged_candidates_test['normalized_all_items_co_graph_count_1'] = count_sum_array[merged_candidates_test['sess_id']]
merged_candidates_test['normalized_all_items_co_graph_count_1'] = merged_candidates_test['all_items_co_graph_count_1'] / merged_candidates_test['normalized_all_items_co_graph_count_1']

/tmp/ipykernel_579818/3162329703.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates_test['normalized_all_items_co_graph_count_1'] = count_sum_array[merged_candidates_test['sess_id']]
/tmp/ipykernel_579818/3162329703.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates_test['normalized_all_items_co_graph_count_1'] = merged_candidates_test['all_items_co_graph_count_1'] / merged_candidates_test['normalized_all_items_co_graph_count_1']


In [97]:
merged_candidates_feature_test['normalized_all_items_co_graph_count_1'] = merged_candidates_test['normalized_all_items_co_graph_count_1']
merged_candidates_feature_test['all_items_co_graph_count_1'] = merged_candidates_test['all_items_co_graph_count_1']

## uni and dis=1

In [98]:
# max dis = 1 
# co_occurence_dict_bi_dis1 = get_co_occurence_dict(train_sess_item + valid_sess_item + test_sess_item, bidirection=True, weighted=False, max_dis=1)
co_occurence_dict_uni_dis1 = get_co_occurence_dict(train_sess_item + valid_sess_item + test_sess_item, bidirection=False, weighted=False, max_dis=1)

100%|██████████| 4236450/4236450 [01:34<00:00, 44821.91it/s] 


In [99]:
# only one arg, can't use another arg
def get_uni_dis1_test_session_co_graph_candidates(sess_id):
    sess = test_sess_item[sess_id]
    prev_items = set()
    cand_counter = Counter()
    for item in sess:
        if item in co_occurence_dict_uni_dis1 and item not in prev_items:
            cand_counter = cand_counter + co_occurence_dict_uni_dis1[item]
            prev_items.add(item) # one time for every item
    for item in sess:
        if item in cand_counter:
            cand_counter.pop(item) # remove history items 
    
    return cand_counter

In [100]:
test_co_graph_candidates = TFDataset.from_dict({'sess_id' : list(range(test_sess_data.shape[0]))})

In [101]:
# about 1 mins
sess_ids = list(range(len(test_sess_item)))
pool = multiprocessing.Pool(10)
uni_dis1_test_sessions_counter = pool.map(get_uni_dis1_test_session_co_graph_candidates, sess_ids)

In [102]:
test_co_graph_candidates, len(uni_dis1_test_sessions_counter)

(Dataset({
     features: ['sess_id'],
     num_rows: 316971
 }),
 316971)

In [103]:
all_items_co_graph_count_list = []
for row in tqdm(merged_candidates_test.itertuples(), total=merged_candidates_test.shape[0]):
    all_items_co_graph_count_list.append(uni_dis1_test_sessions_counter[row.sess_id][row.product])
assert len(all_items_co_graph_count_list) == merged_candidates_test.shape[0]
merged_candidates_test['all_items_co_graph_count_2'] = all_items_co_graph_count_list

100%|██████████| 69428431/69428431 [05:26<00:00, 212734.94it/s]
/tmp/ipykernel_579818/1504731075.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates_test['all_items_co_graph_count_2'] = all_items_co_graph_count_list


In [104]:
count_sum_array = merged_candidates_test.groupby(by='sess_id')['all_items_co_graph_count_2'].sum().to_numpy()
assert len(count_sum_array[merged_candidates_test['sess_id']]) == merged_candidates_test.shape[0]
merged_candidates_test['normalized_all_items_co_graph_count_2'] = count_sum_array[merged_candidates_test['sess_id']]
merged_candidates_test['normalized_all_items_co_graph_count_2'] = merged_candidates_test['all_items_co_graph_count_2'] / merged_candidates_test['normalized_all_items_co_graph_count_2']

/tmp/ipykernel_579818/2112424902.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates_test['normalized_all_items_co_graph_count_2'] = count_sum_array[merged_candidates_test['sess_id']]
/tmp/ipykernel_579818/2112424902.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates_test['normalized_all_items_co_graph_count_2'] = merged_candidates_test['all_items_co_graph_count_2'] / merged_candidates_test['normalized_all_items_co_graph_count_2']


In [105]:
merged_candidates_feature_test['normalized_all_items_co_graph_count_2'] = merged_candidates_test['normalized_all_items_co_graph_count_2']
merged_candidates_feature_test['all_items_co_graph_count_2'] = merged_candidates_test['all_items_co_graph_count_2']

In [106]:
cast_dtype(merged_candidates_feature_test)
merged_candidates_feature_test.to_parquet(merged_candidates_feature_test_path, engine='pyarrow')

In [109]:
merged_candidates_feature_test

,sess_id,sess_locale,product,sasrec_scores_2,sasrec_normalized_scores_2,gru4rec_scores,gru4rec_normalized_scores,product_freq,sess_avg_price,product_price,...,sasrec_normalized_scores_3,normalized_all_items_co_graph_count_0,all_items_co_graph_count_0,seqmlp_scores,seqmlp_normalized_scores,desc_BM25_scores,normalized_all_items_co_graph_count_1,all_items_co_graph_count_1,normalized_all_items_co_graph_count_2,all_items_co_graph_count_2
0,0,DE,4088833651,0.000000,2.975813e-09,0.000000,1.580065e-09,828,25.195269,36.761604,...,2.622550e-09,0.000000,0,0.000000,2.554478e-10,0.000000,0.000000,0.000000,0.000000,0
1,0,DE,B000H6W2GW,0.000000,2.975813e-09,0.000000,1.580065e-09,875,25.195269,36.761604,...,2.622550e-09,0.000000,0,0.000000,2.554478e-10,0.000000,0.000000,0.000000,0.000000,0
2,0,DE,B000JG2RAG,7.665308,6.347557e-06,8.104032,5.226502e-06,24,25.195269,23.190001,...,1.894552e-05,0.000000,0,8.786958,1.672744e-06,67.792648,0.000000,0.000000,0.000000,0
3,0,DE,B000RYSOUW,-2.951060,1.555882e-10,-2.857798,9.068785e-11,5,25.195269,6.900000,...,5.083796e-10,0.000000,0,-3.325048,9.188664e-12,170.360580,0.000000,0.000000,0.000000,0
4,0,DE,B000UGZVQM,3.977920,1.589257e-07,4.688567,1.717488e-07,4,25.195269,21.990000,...,3.784811e-07,0.000000,0,5.540127,6.506522e-08,71.169296,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69428426,316970,UK,B0BJCTH4NH,11.327528,1.041200e-04,10.629994,3.818184e-04,74,16.950001,5.800000,...,1.849500e-05,0.010237,16,11.838901,9.762144e-04,164.803131,0.002540,1.285714,0.003086,1
69428427,316970,UK,B0BJTQQWLG,5.604142,3.403292e-07,6.052083,3.923694e-06,6,16.950001,9.880000,...,5.044600e-07,0.000640,1,4.890683,9.375031e-07,303.665985,0.000494,0.250000,0.000000,0
69428428,316970,UK,B0BJV3RL4H,9.146974,1.176336e-05,7.667603,1.973815e-05,7,16.950001,22.097065,...,1.265775e-06,0.000640,1,10.187823,1.872800e-04,226.131516,0.001976,1.000000,0.003086,1
69428429,316970,UK,B0BK7SPC84,-10.383047,3.879279e-14,-6.356799,1.601719e-11,0,16.950001,5.960000,...,6.057512e-15,0.000000,0,-4.160688,1.099036e-10,312.603607,0.000000,0.000000,0.000000,0


In [108]:
merged_candidates_feature_test.query('sess_id==300000').sort_values(by=['sasrec_normalized_scores_3'], ascending=False)[['sasrec_scores_3', 'sasrec_normalized_scores_3', 'all_items_co_graph_count_0', 'all_items_co_graph_count_1', 'all_items_co_graph_count_2']]

,sasrec_scores_3,sasrec_normalized_scores_3,all_items_co_graph_count_0,all_items_co_graph_count_1,all_items_co_graph_count_2
65675691,18.784042,7.110926e-01,141,64.666664,41
65675602,17.250706,1.534640e-01,127,40.392857,27
65675649,16.779135,9.576479e-02,59,20.842857,7
65675546,15.148463,1.875056e-02,43,11.941667,6
65675714,14.737875,1.243649e-02,29,5.916667,3
...,...,...,...,...,...
65675666,-6.248106,9.563178e-12,0,0.000000,0
65675565,-6.320740,8.893195e-12,0,0.000000,0
65675554,-6.321655,8.885057e-12,0,0.000000,0
65675610,-8.322851,1.201025e-12,0,0.000000,0
